# Warehouse Storage Location Optimization

This notebook analyzes order picking data from a footwear warehouse and optimizes storage locations based on ABC classification and pick frequency.

## Project Overview
- **Objective**: Optimize warehouse storage layout to minimize travel distance
- **Method**: ABC analysis and distance-based allocation
- **Dataset**: Order picking records from footwear manufacturing warehouse


## 1. Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

### Load Data from ZIP Archive

In [ ]:
# Define paths
ZIP_FILE = 'Order Picking Dataset from a Warehouse of a Footwear Manufacturing Company.zip'
DATA_PATH = 'Order Picking Dataset from a Warehouse of a Footwear Manufacturing Company/'

def load_csv_from_zip(zip_file, internal_path, separator=';'):
    """Load CSV file from within a ZIP archive."""
    with zipfile.ZipFile(zip_file, 'r') as z:
        with z.open(internal_path) as f:
            return pd.read_csv(f, sep=separator)

# Load main datasets
print("Loading datasets...")
df_orders = load_csv_from_zip(ZIP_FILE, DATA_PATH + 'Customer_Order.csv')
df_products = load_csv_from_zip(ZIP_FILE, DATA_PATH + 'Product.csv')
df_storage = load_csv_from_zip(ZIP_FILE, DATA_PATH + 'Storage_Location.csv')

print(f"Orders loaded: {len(df_orders):,} records")
print(f"Products loaded: {len(df_products):,} items")
print(f"Storage locations loaded: {len(df_storage):,} positions")

## 2. Data Exploration

In [ ]:
# Display basic information
print("=" * 50)
print("CUSTOMER ORDERS")
print("=" * 50)
print(df_orders.info())
print("\nFirst rows:")
display(df_orders.head())

print("\n" + "=" * 50)
print("PRODUCTS")
print("=" * 50)
print(df_products.info())
display(df_products.head())

print("\n" + "=" * 50)
print("STORAGE LOCATIONS")
print("=" * 50)
print(df_storage.info())
display(df_storage.head())

### Merge Datasets

In [ ]:
# Merge orders with product information
df_merged = df_orders.merge(
    df_products[['Reference', 'ABCCOD', 'Sector']], 
    on='Reference', 
    how='left'
)

print(f"Merged dataset: {len(df_merged):,} records")
print(f"\nMissing ABC codes: {df_merged['ABCCOD'].isna().sum()}")
display(df_merged.head())

## 3. ABC Analysis

Calculate pick frequency and classify products using ABC analysis principles.

In [ ]:
# Calculate pick frequency by reference
pick_frequency = df_merged.groupby('Reference').agg({
    'orderNumber': 'count',
    'quantity (units)': 'sum',
    'ABCCOD': 'first',
    'Sector': 'first'
}).rename(columns={'orderNumber': 'pick_count'}).reset_index()

# Sort by pick frequency
pick_frequency = pick_frequency.sort_values('pick_count', ascending=False)

print("Pick Frequency Statistics:")
print(pick_frequency['pick_count'].describe())
print(f"\nTop 10 Most Picked Items:")
display(pick_frequency.head(10))

### Visualize ABC Distribution

In [ ]:
# Plot ABC distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Count by ABC class
abc_counts = pick_frequency['ABCCOD'].value_counts()
axes[0].bar(abc_counts.index, abc_counts.values, color=['#FF6B6B', '#4ECDC4', '#95E1D3'])
axes[0].set_title('Product Count by ABC Class', fontsize=14, fontweight='bold')
axes[0].set_xlabel('ABC Class')
axes[0].set_ylabel('Number of Products')
axes[0].grid(axis='y', alpha=0.3)

# Pick frequency by ABC class
abc_picks = pick_frequency.groupby('ABCCOD')['pick_count'].sum()
axes[1].bar(abc_picks.index, abc_picks.values, color=['#FF6B6B', '#4ECDC4', '#95E1D3'])
axes[1].set_title('Total Picks by ABC Class', fontsize=14, fontweight='bold')
axes[1].set_xlabel('ABC Class')
axes[1].set_ylabel('Total Pick Count')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nABC Class Statistics:")
print(pick_frequency.groupby('ABCCOD').agg({
    'Reference': 'count',
    'pick_count': ['sum', 'mean', 'median']
}))

## 4. Storage Location Analysis

Analyze current storage locations and calculate distances from depot.

In [ ]:
# Calculate distances from depot (0,0)
df_storage['distance_from_depot'] = np.sqrt(
    df_storage['X']**2 + df_storage['Y']**2
)

# Sort by distance
df_storage_sorted = df_storage.sort_values('distance_from_depot')

print("Storage Location Statistics:")
print(df_storage['distance_from_depot'].describe())

print(f"\nClosest locations to depot:")
display(df_storage_sorted.head(10))

print(f"\nFarthest locations from depot:")
display(df_storage_sorted.tail(10))

## 5. Storage Optimization

Assign storage locations based on ABC classification:
- **Class A** (high frequency): Closest locations
- **Class B** (medium frequency): Medium distance
- **Class C** (low frequency): Farthest locations

In [ ]:
# Count products per ABC class
abc_distribution = pick_frequency['ABCCOD'].value_counts().sort_index()
print("Products per ABC class:")
print(abc_distribution)

# Allocate storage positions
total_positions = len(df_storage_sorted)
positions_A = abc_distribution.get('A', 0)
positions_B = abc_distribution.get('B', 0)
positions_C = abc_distribution.get('C', 0)

# Assign zones
zone_A = df_storage_sorted.iloc[:positions_A].copy()
zone_B = df_storage_sorted.iloc[positions_A:positions_A + positions_B].copy()
zone_C = df_storage_sorted.iloc[positions_A + positions_B:positions_A + positions_B + positions_C].copy()

zone_A['assigned_class'] = 'A'
zone_B['assigned_class'] = 'B'
zone_C['assigned_class'] = 'C'

# Combine optimized layout
df_optimized = pd.concat([zone_A, zone_B, zone_C], ignore_index=True)

print(f"\nOptimized allocation:")
print(f"Zone A: {len(zone_A)} positions (avg distance: {zone_A['distance_from_depot'].mean():.2f}m)")
print(f"Zone B: {len(zone_B)} positions (avg distance: {zone_B['distance_from_depot'].mean():.2f}m)")
print(f"Zone C: {len(zone_C)} positions (avg distance: {zone_C['distance_from_depot'].mean():.2f}m)")

### Calculate Travel Distance Savings

In [ ]:
# Simulation parameters
WALKING_SPEED = 1.2  # m/s
PICK_TIME = 15  # seconds per item

# Calculate current state (random allocation assumption)
current_avg_distance = df_storage['distance_from_depot'].mean()
total_picks = pick_frequency['pick_count'].sum()

# Current state metrics
current_distance = current_avg_distance * total_picks * 2  # Round trip
current_time_hours = (current_distance / WALKING_SPEED + total_picks * PICK_TIME) / 3600

# Optimized state metrics
optimized_distance = (
    zone_A['distance_from_depot'].mean() * abc_distribution.get('A', 0) * pick_frequency[pick_frequency['ABCCOD'] == 'A']['pick_count'].sum() * 2 +
    zone_B['distance_from_depot'].mean() * abc_distribution.get('B', 0) * pick_frequency[pick_frequency['ABCCOD'] == 'B']['pick_count'].sum() * 2 +
    zone_C['distance_from_depot'].mean() * abc_distribution.get('C', 0) * pick_frequency[pick_frequency['ABCCOD'] == 'C']['pick_count'].sum() * 2
) / total_picks

optimized_distance_total = optimized_distance * total_picks
optimized_time_hours = (optimized_distance_total / WALKING_SPEED + total_picks * PICK_TIME) / 3600

# Calculate improvements
distance_saved = current_distance - optimized_distance_total
time_saved_hours = current_time_hours - optimized_time_hours
improvement_pct = (distance_saved / current_distance) * 100

print("\n" + "=" * 60)
print("OPTIMIZATION RESULTS")
print("=" * 60)
print(f"\nCurrent System:")
print(f"  Total travel distance: {current_distance:,.0f} m")
print(f"  Total time required: {current_time_hours:,.1f} hours")

print(f"\nOptimized System:")
print(f"  Total travel distance: {optimized_distance_total:,.0f} m")
print(f"  Total time required: {optimized_time_hours:,.1f} hours")

print(f"\nImprovements:")
print(f"  Distance saved: {distance_saved:,.0f} m ({improvement_pct:.1f}%)")
print(f"  Time saved: {time_saved_hours:,.1f} hours")
print(f"  Efficiency gain: {improvement_pct:.1f}%")
print("=" * 60)

### Visualize Optimization Results

In [ ]:
# Create comparison visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Distance comparison
metrics = ['Current', 'Optimized']
distances = [current_distance/1000, optimized_distance_total/1000]  # Convert to km
colors = ['#FF6B6B', '#4ECDC4']

bars1 = axes[0].bar(metrics, distances, color=colors, width=0.6)
axes[0].set_title('Total Travel Distance Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Distance (km)')
axes[0].grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f} km',
                ha='center', va='bottom', fontweight='bold')

# Time comparison
times = [current_time_hours, optimized_time_hours]
bars2 = axes[1].bar(metrics, times, color=colors, width=0.6)
axes[1].set_title('Total Time Required Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Time (hours)')
axes[1].grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars2:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f} hrs',
                ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Conclusions and Recommendations

### Key Findings:
1. **Significant efficiency gains** through ABC-based storage allocation
2. **High-frequency items** (Class A) benefit most from proximity to depot
3. **Reduced travel distance** leads to faster order fulfillment

### Recommendations:
1. Implement the optimized storage layout
2. Regularly review ABC classifications (quarterly)
3. Monitor seasonal variations in pick frequency
4. Consider dynamic reallocation for trending products
5. Train staff on new storage locations

### Next Steps:
- Pilot test in one warehouse section
- Measure actual performance improvements
- Adjust allocation based on real-world results
- Expand to full warehouse implementation